## Import necessary modules

In [1]:
from DataHandler.DataHandler import DataHandler
from DataHandler.ModelPreparator import ModelPreparator
from model.ModelHandler import Bert_Trainer_Evaluator

### Initialize DataHandler

In [2]:
datahandler=DataHandler()

### Read the input file

In [3]:
df = datahandler.read_data("data\steam_reviews.parquet")

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\rapid\AppData\Local\Temp\ipykernel_28960\3046307167.py:1: SyntaxWarning: invalid escape sequence '\s'
  df = datahandler.read_data("data\steam_reviews.parquet")


### Preprocess the input data and choose the dataset size

In [4]:
df = datahandler.preprocess_data(df=df)
df = datahandler.choose_dataset_size(df=df, row_number=500)

e:\Uni\Master\3. Semester\NLP\Project\steamrep\DataHandler\DataHandler.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['cleaned_review'] = df_filtered['review'].apply(
e:\Uni\Master\3. Semester\NLP\Project\steamrep\DataHandler\DataHandler.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['cleaned_review'] = df_filtered['cleaned_review'].str.lower()


### Initialize the ModelPreparator

In [5]:
modelpreparator = ModelPreparator()

### Add the column with sarcasm analysis

In [6]:
df = modelpreparator.add_sarcasm_score_to_review(df=df, batch_size=16)

Processing reviews:   0%|          | 0/32 [00:00<?, ?it/s]C:\Users\rapid\AppData\Roaming\Python\Python312\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:545.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Processing reviews: 100%|██████████| 32/32 [00:07<00:00,  4.51it/s]


### Make a train test split and tokenize for both columns

In [7]:
cols_mapping = {"tokenized": "cleaned_review",
                "tokenized_sarcasm": "cleaned_review_sarcasm"}
# brauchen wir prepare_for_bert überhaupt?
df = modelpreparator.prepare_for_bert(df=df, cols_mapping=cols_mapping)
train_df, val_df = modelpreparator.make_train_test_split(df=df)

Training Set: 400 Reviews
Validation Set: 100 Reviews


### Train Bert for both columns (i.e. with and without sarcasm score) and compare results. 

In [8]:
review_columns = ['cleaned_review', 'cleaned_review_sarcasm']
bert_handler = Bert_Trainer_Evaluator(train_df, val_df, review_columns)
bert_handler.train(epochs=3)

Using device: cuda


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\rapid\AppData\Roaming\Python\Python312\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to d

Training for column: cleaned_review
Epoch 1/3 for cleaned_review


Training cleaned_review: 100%|██████████| 25/25 [00:53<00:00,  2.14s/it]


Average training loss for cleaned_review: 0.48222267627716064
Epoch 2/3 for cleaned_review


...

Average training loss for cleaned_review_sarcasm: 0.40324351787567136
Epoch 3/3 for cleaned_review_sarcasm


Training cleaned_review_sarcasm: 100%|██████████| 25/25 [01:58<00:00,  4.75s/it]


Average training loss for cleaned_review_sarcasm: 0.3360268670320511


Validating cleaned_review_sarcasm: 100%|██████████| 7/7 [00:02<00:00,  2.98it/s]


Validation Metrics for cleaned_review_sarcasm:
  Accuracy: 0.8800
  Precision: 0.9294
  Recall: 0.9294
  F1-Score: 0.9294
  Confusion Matrix:
[[ 9  6]
 [ 6 79]]
  Validation Loss: 0.2827

Comparison of Performance Metrics:
Column: cleaned_review
  Accuracy: 0.8800
  Precision: 0.8763
  Recall: 1.0000
  F1-Score: 0.9341
  Validation Loss: 0.3055
  Confusion Matrix:
[[ 3 12]
 [ 0 85]]

Column: cleaned_review_sarcasm
  Accuracy: 0.8800
  Precision: 0.9294
  Recall: 0.9294
  F1-Score: 0.9294
  Validation Loss: 0.2827
  Confusion Matrix:
[[ 9  6]
 [ 6 79]]

